In [ ]:
import pandas as pd
import os
import re
from sentence_transformers import SentenceTransformer

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import codecs

In [ ]:
df = pd.read_csv("corona_tweets_Feb2020.csv") #the csv file has been modified according to the most popular tweets i.e. arranged in order of number of likes or "favorite count"

In [ ]:
df.columns

Index(['user_id', 'status_id', 'created_at', 'text', 'source',
       'display_text_width', 'reply_to_status_id', 'reply_to_user_id',
       'reply_to_screen_name', 'is_quote', 'is_retweet', 'favorite_count',
       'retweet_count', 'quote_count', 'reply_count'],
      dtype='object')

In [ ]:
df = df[0:100] #top 100 tweets based on number of likes are selected

In [ ]:
df

,user_id,status_id,created_at,text,source,display_text_width,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,is_retweet,favorite_count,retweet_count,quote_count,reply_count
0,1.651523e+09,1.230000e+18,2/27/2020 6:36,As an MD PhD who spent years developing an Ebo...,Twitter for iPhone,172,NaN,NaN,NaN,True,False,91,23,NaN,NaN
1,1.115875e+09,1.230000e+18,2/27/2020 6:33,Denmark Norway both report first coronavirus ...,Twitter Web App,163,NaN,NaN,NaN,False,False,32,11,NaN,NaN
2,8.777542e+07,1.230000e+18,2/27/2020 6:30,EverydayHero Salute to the people who still s...,TeamSight Publisher,252,NaN,NaN,NaN,False,False,29,8,NaN,NaN
3,3.393515e+08,1.230000e+18,2/27/2020 6:36,As Mike Pence pulls out his to the rescue scht...,Twitter for iPhone,278,NaN,NaN,NaN,False,False,26,10,NaN,NaN
4,6.782762e+06,1.230000e+18,2/27/2020 6:31,I have no idea why anyone is traveling at all ...,Twitter Web App,232,NaN,NaN,NaN,True,False,24,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.858727e+07,1.230000e+18,2/27/2020 6:35,I have yet to see the West fight the coronavir...,Twitter Web App,133,NaN,NaN,NaN,True,False,1,0,NaN,NaN
96,1.079911e+09,1.230000e+18,2/27/2020 6:35,World 'on brink of coronavirus pandemic' All t...,EcoInternet3,181,NaN,NaN,NaN,False,False,1,1,NaN,NaN
97,1.190000e+18,1.230000e+18,2/27/2020 6:35,First cases of coronavirus confirmed in DENMAR...,Twitter Web App,82,NaN,NaN,NaN,False,False,1,2,NaN,NaN
98,5.473719e+07,1.230000e+18,2/27/2020 6:35,'The whole Australian community needs to work ...,Hootsuite Inc.,275,NaN,NaN,NaN,False,False,1,1,NaN,NaN


In [ ]:

titles = df['text']



In [ ]:
df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', case=False,regex=True)
df['text'] = df['text'].str.replace(r'<[^<>]*>', ' ', regex=True)
df['text'] = df['text'].str.replace('[^\w\s]', ' ', regex=True)
df['text'] = df['text'].str.replace('\n', '')


In [ ]:
print(df.dtypes)

user_id                 float64
status_id               float64
created_at               object
text                     object
source                   object
display_text_width        int64
reply_to_status_id      float64
reply_to_user_id        float64
reply_to_screen_name     object
is_quote                   bool
is_retweet                 bool
favorite_count            int64
retweet_count             int64
quote_count             float64
reply_count             float64
dtype: object


In [ ]:
df['text'] = df['text'].str.replace(u'\xa0', u' ') #It replaces all instances of the non-breaking space character (\xa0) in the text column of the DataFrame df with a regular space (' ').

In [ ]:
docs = df['text'].tolist()


In [ ]:
len(docs)

100

# 1. Text Embedding (Stella 400M V5)

In [ ]:
test = docs[:2]

In [ ]:
test

['As an MD PhD who spent years developing an Ebola treatment I unfortunately agree with this prediction CoronavirusOutbreak COVID19 coronavirus CoronavirusPandemic ',
 ' Denmark Norway both report first coronavirus cases A Danish male tested positive for COVID19 after returning from Italy on Monday according to local media ']

In [ ]:

# query_prompt_name = "s2p_query"

# # ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)


In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2", trust_remote_code=True)



In [ ]:
test = ['apple', 'I love you']

In [ ]:
# Test. For the entire data, this takes a lot of time
doc_embeddings = model.encode(test)

print(doc_embeddings.shape)


(2, 384)


In [ ]:
doc_embeddings

array([[-6.13846118e-03,  3.10117584e-02,  6.47935644e-02,
         1.09415101e-02,  5.26716514e-03, -4.74764444e-02,
         8.12030435e-02,  2.89809797e-02,  6.67620078e-02,
         3.03004012e-02,  5.74649386e-02, -8.62350781e-03,
         1.32275815e-03,  3.99174896e-04, -1.88430324e-02,
        -2.57937964e-02, -1.30420933e-02, -5.26249632e-02,
        -5.82925119e-02, -2.58992277e-02, -3.33736502e-02,
         2.45678779e-02, -5.22664748e-03,  2.30059847e-02,
         3.28607149e-02,  7.50217736e-02,  5.80176199e-03,
        -1.49586024e-02, -2.87529211e-02, -1.18551917e-01,
        -3.93217392e-02, -5.13875633e-02,  7.66182840e-02,
         4.84036840e-02, -3.02563850e-02, -9.14341658e-02,
         5.11820838e-02, -9.64961760e-03, -2.15108134e-02,
        -7.17740878e-02, -6.32243156e-02, -1.76688787e-02,
         2.80814916e-02,  9.00470689e-02,  1.94179956e-02,
         5.45446854e-03,  4.83017936e-02,  8.49085953e-03,
         2.77506020e-02,  9.65605751e-02,  2.63027102e-0

In [ ]:
# docs = docs.apply(lambda x: textwrap.shorten(x, width=10, placeholder="..."))
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

In [ ]:
embedding = {}
i=0

embedding = model.encode(docs)

# for item in docs:
#     print (str("Trying...") + str(i))
#     try:
#         embedding[i] = model.encode(item)
#         print(str(embedding[i]))
#         i = i+1
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")



In [ ]:
embedding

array([[ 0.00949351,  0.01995499,  0.01909441, ..., -0.08243945,
        -0.01260356,  0.02464234],
       [-0.03298308,  0.11509782, -0.03246538, ..., -0.05823608,
         0.03584712,  0.04790984],
       [-0.02915162,  0.09725288, -0.05441043, ..., -0.01055765,
        -0.02323569,  0.01340001],
       ...,
       [ 0.01615389,  0.08916029, -0.0210476 , ..., -0.04117454,
         0.07913782,  0.0840958 ],
       [-0.02323654,  0.04789267, -0.07746919, ..., -0.0798737 ,
        -0.00096014,  0.05984626],
       [ 0.00613427, -0.03324327,  0.04409526, ..., -0.0917344 ,
         0.00907943,  0.0511189 ]], dtype=float32)

In [ ]:
i=0

for em in embedding:
    embedding[i] = em.tolist()

In [ ]:
embedding = np.array(embedding)

In [ ]:
embedding

array([[ 0.00613427, -0.03324327,  0.04409526, ..., -0.0917344 ,
         0.00907943,  0.0511189 ],
       [-0.03298308,  0.11509782, -0.03246538, ..., -0.05823608,
         0.03584712,  0.04790984],
       [-0.02915162,  0.09725288, -0.05441043, ..., -0.01055765,
        -0.02323569,  0.01340001],
       ...,
       [ 0.01615389,  0.08916029, -0.0210476 , ..., -0.04117454,
         0.07913782,  0.0840958 ],
       [-0.02323654,  0.04789267, -0.07746919, ..., -0.0798737 ,
        -0.00096014,  0.05984626],
       [ 0.00613427, -0.03324327,  0.04409526, ..., -0.0917344 ,
         0.00907943,  0.0511189 ]], dtype=float32)

In [ ]:
data = pd.DataFrame(embedding)
data.to_csv("embedding_outcome.csv", header=False, index=False)

In [ ]:
np.savetxt("embedding_array.csv", data, delimiter=",")

In [ ]:
print(embedding.shape)

(100, 384)


In [ ]:
embedding = pd.read_csv("embedding_array.csv", header=None)

In [ ]:
embedding = embedding.to_numpy()

# 2. UMAP (Dimensionality Reduction)

In [ ]:
from umap import UMAP

# random_state=42 to prevent any stochastic behavior
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=3)

# 3. HDB Scan to decide the number of topics

In [ ]:
from hdbscan import HDBSCAN

# indirectly controlloing the number of topics by setting the minimum size of the clsuter
hdbscan_model = HDBSCAN(min_cluster_size=3, metric='euclidean', cluster_selection_method='eom')

# 4. Stop Words in Topic Models

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

my_additional_stop_words = ["meetup", "meet", "gathering", "group", "join"]
vectorizer_model = CountVectorizer(stop_words = list(text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)), min_df=2, ngram_range=(1, 2))

In [ ]:
text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [ ]:
!pip install bertopic

# 5. Using OpenAI to Label Each Cluster (topic model is KeyBERT)

In [ ]:
import openai
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
# pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-o3-mini
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
client = openai.OpenAI(api_key="***")
openai_model = OpenAI(client, model="o3-mini", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "OpenAI": openai_model,
    "MMR": mmr_model,
    # "POS": pos_model
}

In [ ]:
docs = docs[0:101]

In [ ]:
docs

['As an MD PhD who spent years developing an Ebola treatment I unfortunately agree with this prediction CoronavirusOutbreak COVID19 coronavirus CoronavirusPandemic ',
 ' Denmark Norway both report first coronavirus cases A Danish male tested positive for COVID19 after returning from Italy on Monday according to local media ',
 ' EverydayHero Salute to the people who still stick to their posts A 60 year old chef cried when his efforts were not appreciated by his colleagues He is not a superman We should appreciate others  efforts in fighting the novel coronavirus COVID19 ',
 'As Mike Pence pulls out his to the rescue schtick and feigns competence Trump stands back and hopes to God a miracle will solve this crisis With a global pandemic looming Trump s both furious and fearful the coronavirus may foil all his evil plans ',
 'I have no idea why anyone is traveling at all There are no outright travel bans and everyone does business with China This isn t that difficult but proves our god is

In [ ]:
docs = [str(element) for element in docs]

In [ ]:
docs

['As an MD PhD who spent years developing an Ebola treatment I unfortunately agree with this prediction CoronavirusOutbreak COVID19 coronavirus CoronavirusPandemic ',
 ' Denmark Norway both report first coronavirus cases A Danish male tested positive for COVID19 after returning from Italy on Monday according to local media ',
 ' EverydayHero Salute to the people who still stick to their posts A 60 year old chef cried when his efforts were not appreciated by his colleagues He is not a superman We should appreciate others  efforts in fighting the novel coronavirus COVID19 ',
 'As Mike Pence pulls out his to the rescue schtick and feigns competence Trump stands back and hopes to God a miracle will solve this crisis With a global pandemic looming Trump s both furious and fearful the coronavirus may foil all his evil plans ',
 'I have no idea why anyone is traveling at all There are no outright travel bans and everyone does business with China This isn t that difficult but proves our god is

# 6. Training

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)



In [ ]:
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
import gc
# Trigger garbage collection manually
gc.collect()

9039

In [ ]:
topics, probs = topic_model.fit_transform(docs, embedding)

2025-04-08 01:00:44,019 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-08 01:01:04,840 - BERTopic - Dimensionality - Completed ✓
2025-04-08 01:01:04,849 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-08 01:01:04,887 - BERTopic - Cluster - Completed ✓
2025-04-08 01:01:04,904 - BERTopic - Representation - Fine-tuning topics using representation models.
100%|██████████| 11/11 [00:55<00:00,  5.03s/it]
2025-04-08 01:02:05,402 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,OpenAI,MMR,Representative_Docs
0,-1,26,-1_coronavirus_coronavirusupdates_new_coronavi...,"[coronavirus, coronavirusupdates, new, coronav...","[coronavirusupdates, coronavirus facts, spread...",[Coronavirus Outbreak Updates],"[coronavirus, coronavirusupdates, coronaviruso...",[ Reuters Zero sensationalism and 100 Facts we...
1,0,13,0_china_reported_cases_coronavirus,"[china, reported, cases, coronavirus, infectio...","[new coronavirus, coronavirus coronavirus, cor...",[Coronavirus Reporting and Spread],"[china, cases, coronavirus, coronavirusoutbrea...",[ Coronavirus is especially lethal for people ...
2,1,11,1_confirmed_case_coronavirus case_covid19,"[confirmed, case, coronavirus case, covid19, c...","[coronavirus case, case coronavirus, coronavir...",[First Confirmed COVID-19 Cases],"[coronavirus case, coronavirus, positive covid...",[Denmark Norway both report first coronavirus ...
3,2,10,2_ban_travel_iran_non,"[ban, travel, iran, non, coronavirus, china, c...","[coronavirus fears, coronavirus, spread corona...",[Global COVID Travel Bans],"[ban, iran, coronavirus, fears, coronaviruspak...",[UPDATE MoH advises Kenyans against non essent...
4,3,9,3_trump_people_president_virus,"[trump, people, president, virus, coronavirus,...","[new coronavirus, coronavirus, coronavirus cov...",[Trump coronavirus response],"[trump, virus, coronavirus, authorities, medic...",[Trump response to drsanjaygupta ass chief neu...
5,4,7,4_school_high_coronavirus concerns_wait,"[school, high, coronavirus concerns, wait, con...","[coronavirus concerns, case coronavirus, coron...",[Coronavirus High School Closures],"[school, coronavirus concerns, concerns, close...",[A resident of Solano County California has no...
6,5,6,5_going_covid19_community_sarscov2,"[going, covid19, community, sarscov2, care, cr...","[covid19 coronavirus, outbreak coronavirus, co...",[COVID-19 Outbreak and Community Spread],"[covid19, sarscov2, community spread, outbreak...",[We can now confirm that a patient UCDavisMedC...
7,6,5,6_health_coronavirusoutbreak_coronavirusoutbre...,"[health, coronavirusoutbreak, coronavirusoutbr...","[covid19 coronavirus, coronavirusoutbreak covi...",[Coronavirus Pandemic Health Control],"[coronavirusoutbreak, coronavirusoutbreak covi...",[As an MD PhD who spent years developing an Eb...
8,7,5,7_facts_100 facts_100_chief,"[facts, 100 facts, 100, chief, contained, cove...","[outbreak coronavirus, coronavirus fears, cove...",[COVID-19 Facts and Anti-Racism],"[100 facts, chief, covering coronavirus, coron...",[To defeat this outbreak of coronavirus COVID1...
9,8,4,8_face_coronaviruspakistan_completely_deputy,"[face, coronaviruspakistan, completely, deputy...","[coronaviruspakistan, coronavirus coronavirus,...",[Coronavirus Face Mask Measures],"[face, coronaviruspakistan, public, prime mini...",[One coronavirus side effect anti infection ma...


In [ ]:
topic_model.get_topic(1, full=True)

{'Main': [('confirmed', np.float64(0.1631527977454931)),
  ('case', np.float64(0.16221911330179772)),
  ('coronavirus case', np.float64(0.1478686530796146)),
  ('covid19', np.float64(0.11520885996297396)),
  ('cases', np.float64(0.10814607553453183)),
  ('coronavirus', np.float64(0.08976874389516222)),
  ('positive covid19', np.float64(0.08749522058374243)),
  ('breaking', np.float64(0.08749522058374243)),
  ('local', np.float64(0.07980882659313285)),
  ('case coronavirus', np.float64(0.07980882659313285))],
 'KeyBERT': [('coronavirus case', np.float32(0.7062783)),
  ('case coronavirus', np.float32(0.6898323)),
  ('coronavirus', np.float32(0.64138)),
  ('coronavirus covid19', np.float32(0.6221951)),
  ('coronavirusoutbreak coronavirus', np.float32(0.60198087)),
  ('coronavirusoutbreak', np.float32(0.5776783)),
  ('positive covid19', np.float32(0.437721)),
  ('tested positive', np.float32(0.4011073)),
  ('covid2019', np.float32(0.39714605)),
  ('covid19', np.float32(0.37794757))],
 'Ope

In [ ]:
# ChatGPT's labels
chatgpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
topic_model.set_topic_labels(chatgpt_topic_labels)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,OpenAI,MMR,Representative_Docs
0,-1,26,-1_coronavirus_coronavirusupdates_new_coronavi...,Coronavirus Outbreak Updates,"[coronavirus, coronavirusupdates, new, coronav...","[coronavirusupdates, coronavirus facts, spread...",[Coronavirus Outbreak Updates],"[coronavirus, coronavirusupdates, coronaviruso...",[ Reuters Zero sensationalism and 100 Facts we...
1,0,13,0_china_reported_cases_coronavirus,Coronavirus Reporting and Spread,"[china, reported, cases, coronavirus, infectio...","[new coronavirus, coronavirus coronavirus, cor...",[Coronavirus Reporting and Spread],"[china, cases, coronavirus, coronavirusoutbrea...",[ Coronavirus is especially lethal for people ...
2,1,11,1_confirmed_case_coronavirus case_covid19,First Confirmed COVID-19 Cases,"[confirmed, case, coronavirus case, covid19, c...","[coronavirus case, case coronavirus, coronavir...",[First Confirmed COVID-19 Cases],"[coronavirus case, coronavirus, positive covid...",[Denmark Norway both report first coronavirus ...
3,2,10,2_ban_travel_iran_non,Global COVID Travel Bans,"[ban, travel, iran, non, coronavirus, china, c...","[coronavirus fears, coronavirus, spread corona...",[Global COVID Travel Bans],"[ban, iran, coronavirus, fears, coronaviruspak...",[UPDATE MoH advises Kenyans against non essent...
4,3,9,3_trump_people_president_virus,Trump coronavirus response,"[trump, people, president, virus, coronavirus,...","[new coronavirus, coronavirus, coronavirus cov...",[Trump coronavirus response],"[trump, virus, coronavirus, authorities, medic...",[Trump response to drsanjaygupta ass chief neu...
5,4,7,4_school_high_coronavirus concerns_wait,Coronavirus High School Closures,"[school, high, coronavirus concerns, wait, con...","[coronavirus concerns, case coronavirus, coron...",[Coronavirus High School Closures],"[school, coronavirus concerns, concerns, close...",[A resident of Solano County California has no...
6,5,6,5_going_covid19_community_sarscov2,COVID-19 Outbreak and Community Spread,"[going, covid19, community, sarscov2, care, cr...","[covid19 coronavirus, outbreak coronavirus, co...",[COVID-19 Outbreak and Community Spread],"[covid19, sarscov2, community spread, outbreak...",[We can now confirm that a patient UCDavisMedC...
7,6,5,6_health_coronavirusoutbreak_coronavirusoutbre...,Coronavirus Pandemic Health Control,"[health, coronavirusoutbreak, coronavirusoutbr...","[covid19 coronavirus, coronavirusoutbreak covi...",[Coronavirus Pandemic Health Control],"[coronavirusoutbreak, coronavirusoutbreak covi...",[As an MD PhD who spent years developing an Eb...
8,7,5,7_facts_100 facts_100_chief,COVID-19 Facts and Anti-Racism,"[facts, 100 facts, 100, chief, contained, cove...","[outbreak coronavirus, coronavirus fears, cove...",[COVID-19 Facts and Anti-Racism],"[100 facts, chief, covering coronavirus, coron...",[To defeat this outbreak of coronavirus COVID1...
9,8,4,8_face_coronaviruspakistan_completely_deputy,Coronavirus Face Mask Measures,"[face, coronaviruspakistan, completely, deputy...","[coronaviruspakistan, coronavirus coronavirus,...",[Coronavirus Face Mask Measures],"[face, coronaviruspakistan, public, prime mini...",[One coronavirus side effect anti infection ma...


In [ ]:
# Reduce outliers
new_topics = topic_model.reduce_outliers(docs, topics)

# Reduce outliers with pre-calculate embeddings instead
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings", embeddings=embedding)

100%|██████████| 1/1 [00:00<00:00, 116.89it/s]


In [ ]:
topic_model.update_topics(docs, topics=new_topics)

2025-04-08 01:02:05,583 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [ ]:
embedding

array([[ 0.00613427, -0.03324327,  0.04409526, ..., -0.0917344 ,
         0.00907943,  0.0511189 ],
       [-0.03298308,  0.11509782, -0.03246538, ..., -0.05823608,
         0.03584712,  0.04790984],
       [-0.02915162,  0.09725288, -0.05441043, ..., -0.01055765,
        -0.02323569,  0.01340001],
       ...,
       [ 0.01615389,  0.08916029, -0.0210476 , ..., -0.04117454,
         0.07913782,  0.0840958 ],
       [-0.02323654,  0.04789267, -0.07746919, ..., -0.0798737 ,
        -0.00096014,  0.05984626],
       [ 0.00613427, -0.03324327,  0.04409526, ..., -0.0917344 ,
         0.00907943,  0.0511189 ]])

In [ ]:
topic_model.visualize_documents(docs, reduced_embeddings=embedding, custom_labels=True)